# Simple Linear versus Ridge Regression 

Program that illustrates the difference in training and test costs for both linear and ridge regression on the same dataset.

We will use a dataset that is part of the sklearn.dataset package.

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [1]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold

###  Importing the dataset

In [2]:
# Import the boston dataset from sklearn
boston_data = load_boston()

In [3]:
#  Create X and Y variables - X holding the .data and Y holding .target 
X = boston_data.data
y = boston_data.target
print("x shape: ", X.shape)
print("y shape: ", y.shape)
#  Reshape Y to be a rank 2 matrix 
y = y.reshape(X.shape[0], 1)
print("y after reshape: ", y.shape)
# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])
# Printing out the features
print('The features: ', boston_data.feature_names)
# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])
#Observing the first 2 rows of the data
print(X[0:2])


x shape:  (506, 13)
y shape:  (506,)
y after reshape:  (506, 1)
The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


We will also create polynomial features for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. We can also increase the # of degress and see what effect it has on the training and test error. 

In [4]:
# Create a PolynomialFeatures object with degree = 2. 
poly = PolynomialFeatures(degree=2)
X_2 = poly.fit_transform(X)
y_2 = y

In [5]:
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


In [6]:
def get_prediction(X, w):
    yp = X @ w;
    return yp;

In [7]:

def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    xt = X_train.T
    rows = X_train.shape[1]
    I = np.identity(rows)
    alphaI = alpha * I;
    xtx = xt @ X_train
    inv = np.linalg.pinv(xtx + alphaI)
    w = inv @ xt @ y_train
    
    return w

In [8]:
def evaluation_err(X_train, X_test, y_train, y_test, w): 
    
    train_prediction = get_prediction(X_train, w)
    test_prediction = get_prediction(X_test, w)
    train_error = np.mean(np.square(train_prediction - y_train),axis = 0)
    test_error = np.mean(np.square(test_prediction - y_test),axis = 0)
    
    return train_error, test_error

In [9]:

def k_fold_cross_validation(k, X, y, alpha):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        y_train_mean = np.mean(y_train)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean
        # scaling the data matrix
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

          
        w_ridge = get_coeff_ridge_normaleq(X_train, y_train, alpha)
        ridge_train_err, ridge_test_err = evaluation_err(X_train, X_test, y_train, y_test, w_ridge)
        total_E_val_test += ridge_test_err
        total_E_val_train += ridge_train_err
       
    E_val_test = total_E_val_test/k
    E_val_train = total_E_val_train/k
    return  E_val_test, E_val_train

### Degree 1

In [10]:
myx = np.array([[5, 0.5, 2, 0, 4, 8, 4, 6, 2, 2, 2, 4, 5.5]])
print(myx.shape)

(1, 13)


In [11]:
lambdas = np.logspace(1,7, num = 13)
lambdas = np.insert(lambdas, 0, 0, axis = 0)
for lam in lambdas:
    print("lambda: ", lam)
    E_val_test, E_val_train = k_fold_cross_validation(10, X, y, lam)
    print("avg test err: ", E_val_test)
    print("avg train err: ", E_val_train)
    print("*************************************************************************************")

lambda:  0.0
avg test err:  [23.63606861]
avg train err:  [21.80618358]
*************************************************************************************
lambda:  10.0
avg test err:  [23.688583]
avg train err:  [21.89290116]
*************************************************************************************
lambda:  31.622776601683793
avg test err:  [24.0178403]
avg train err:  [22.28544406]
*************************************************************************************
lambda:  100.0
avg test err:  [25.29385255]
avg train err:  [23.72548838]
*************************************************************************************
lambda:  316.22776601683796
avg test err:  [29.45729622]
avg train err:  [28.1665541]
*************************************************************************************
lambda:  1000.0
avg test err:  [39.48949336]
avg train err:  [38.53214873]
*************************************************************************************
lambda:  3162.277660

### Degree 2

In [12]:
for lam in lambdas:
    print("lambda: ", lam)
    E_val_test, E_val_train = k_fold_cross_validation(10, X_2, y_2, lam)
    print("avg test err: ", E_val_test)
    print("avg train err: ", E_val_train)
    print("*************************************************************************************")

lambda:  0.0
avg test err:  [11.85496823]
avg train err:  [5.80882082]
*************************************************************************************
lambda:  10.0
avg test err:  [13.476138]
avg train err:  [10.04905587]
*************************************************************************************
lambda:  31.622776601683793
avg test err:  [15.82960197]
avg train err:  [12.75170627]
*************************************************************************************
lambda:  100.0
avg test err:  [18.98001882]
avg train err:  [16.22269059]
*************************************************************************************
lambda:  316.22776601683796
avg test err:  [22.06869231]
avg train err:  [19.70025365]
*************************************************************************************
lambda:  1000.0
avg test err:  [26.21847559]
avg train err:  [24.28745798]
*************************************************************************************
lambda:  3162.27766

#### we get least error with degree 2 when lambda = 0 i.e. for linear regression 

In [13]:
myx_2 = poly.transform(myx)

print(y_2)
y_mean = np.mean(y)
y_scaled = y - y_mean

# scaling the data matrix
scaler = preprocessing.StandardScaler().fit(X_2)
X_scaled = scaler.transform(X_2)
myx_scaled = scaler.transform(myx_2)
        
curr_w = get_coeff_ridge_normaleq(X_scaled, y_scaled, 0)
myx_prediction = get_prediction(myx_scaled, curr_w)
# predicted_price = scaler.inverse_transform(myx_prediction)
print("using lambda = 0 and degree 2")
print("for x = [5, 0.5, 2, 0, 4, 8, 4, 6, 2, 2, 2, 4, 5.5]")
print("preidicted price: ", myx_prediction + y_mean) # +y_train_mean

[[24. ]
 [21.6]
 [34.7]
 [33.4]
 [36.2]
 [28.7]
 [22.9]
 [27.1]
 [16.5]
 [18.9]
 [15. ]
 [18.9]
 [21.7]
 [20.4]
 [18.2]
 [19.9]
 [23.1]
 [17.5]
 [20.2]
 [18.2]
 [13.6]
 [19.6]
 [15.2]
 [14.5]
 [15.6]
 [13.9]
 [16.6]
 [14.8]
 [18.4]
 [21. ]
 [12.7]
 [14.5]
 [13.2]
 [13.1]
 [13.5]
 [18.9]
 [20. ]
 [21. ]
 [24.7]
 [30.8]
 [34.9]
 [26.6]
 [25.3]
 [24.7]
 [21.2]
 [19.3]
 [20. ]
 [16.6]
 [14.4]
 [19.4]
 [19.7]
 [20.5]
 [25. ]
 [23.4]
 [18.9]
 [35.4]
 [24.7]
 [31.6]
 [23.3]
 [19.6]
 [18.7]
 [16. ]
 [22.2]
 [25. ]
 [33. ]
 [23.5]
 [19.4]
 [22. ]
 [17.4]
 [20.9]
 [24.2]
 [21.7]
 [22.8]
 [23.4]
 [24.1]
 [21.4]
 [20. ]
 [20.8]
 [21.2]
 [20.3]
 [28. ]
 [23.9]
 [24.8]
 [22.9]
 [23.9]
 [26.6]
 [22.5]
 [22.2]
 [23.6]
 [28.7]
 [22.6]
 [22. ]
 [22.9]
 [25. ]
 [20.6]
 [28.4]
 [21.4]
 [38.7]
 [43.8]
 [33.2]
 [27.5]
 [26.5]
 [18.6]
 [19.3]
 [20.1]
 [19.5]
 [19.5]
 [20.4]
 [19.8]
 [19.4]
 [21.7]
 [22.8]
 [18.8]
 [18.7]
 [18.5]
 [18.3]
 [21.2]
 [19.2]
 [20.4]
 [19.3]
 [22. ]
 [20.3]
 [20.5]
 [17.3]
 [18.8]
